In [1]:
import pandas as pd
import os
from pathlib import Path

log_path = os.path.join(Path.cwd().parent, 'logs')
print(log_path)

independent_path = os.path.join(log_path, 'independent')
dependent_path = os.path.join(log_path, 'dependent')

print(independent_path)
print(dependent_path)

/mnt/nvme1/yihaoyuan/Raven/RavenEx/multimodal_emotion_recognition/logs
/mnt/nvme1/yihaoyuan/Raven/RavenEx/multimodal_emotion_recognition/logs/independent
/mnt/nvme1/yihaoyuan/Raven/RavenEx/multimodal_emotion_recognition/logs/dependent


In [2]:
# 2024-12-2 查看非跨被试2分类和4分类结果
binary_res = pd.read_excel(
    os.path.join(independent_path, "2024-11-28_22-55-40", "history_res.xlsx")
)
four_res = pd.read_excel(
    os.path.join(independent_path, "2024-11-28_23-22-25", "history_res.xlsx")
)

print(binary_res.iloc[31, :])
print(four_res.iloc[31, :])

# print(binary_res.iloc[:31, 2])
binary_std = binary_res.iloc[:31, 2].std()
four_std = four_res.iloc[:31, 2].std()
print(binary_std, four_std)


subject         Mean
epoch            NaN
acc         0.906254
loss        0.384739
f1-score    0.901943
cm               NaN
Name: 31, dtype: object
subject         Mean
epoch            NaN
acc         0.558468
loss         1.10841
f1-score    0.518974
cm               NaN
Name: 31, dtype: object
0.08941090511312688 0.07538852967270726


In [3]:
# 2024-12-2 查看跨被试2分类和4分类结果
binary_res = pd.read_excel(
    os.path.join(dependent_path, "2024-11-28_22-56-39", "history_res.xlsx")
)
four_res = pd.read_excel(
    os.path.join(dependent_path, "2024-11-28_23-23-00", "history_res.xlsx")
)

print(binary_res.iloc[10, :])
print(four_res.iloc[10, :])

# print(binary_res.iloc[:10, 2])

binary_std = binary_res.iloc[:10, 2].std()
four_std = four_res.iloc[:10, 2].std()
print(binary_std, four_std)


subject         Mean
epoch            NaN
acc         0.826091
loss         0.46691
f1-score    0.823948
cm               NaN
Name: 10, dtype: object
subject         Mean
epoch            NaN
acc         0.478492
loss        1.227913
f1-score    0.460451
cm               NaN
Name: 10, dtype: object
0.027633928518982713 0.02616577253633707


In [4]:
print(binary_res.iloc[-2,2])
(sum([(i[2] - binary_res.iloc[-2,2]) ** 2 for i in binary_res.iloc[:10, :].values]) / len(binary_res.iloc[:10, :])) ** 0.5

0.8260905760905759


0.02621584644548067

In [5]:
import re
import numpy as np
# 定义一个函数用于解析混淆矩阵字符串并转换为二维数组（numpy数组形式方便后续计算）
def parse_cm(cm_str):
    sub_lists_str = re.sub(r'^\[|\]$', '', cm_str).split('],[')
    cm = np.array([list(map(int, re.findall(r'\d+', sub_list_str))) for sub_list_str in sub_lists_str])
    return cm
test_df = binary_res.copy()
cm = test_df["cm"]
cm.dropna(inplace=True)

# 对DataFrame中的每一个混淆矩阵字符串应用解析函数，得到包含二维数组的新列
cm['cm_parsed'] = cm.apply(parse_cm)

print(cm['cm_parsed'])
# 将解析后的二维数组堆叠起来，方便计算每列平均值
stacked_cm = np.sum(cm['cm_parsed'].values, axis=0)

print(stacked_cm)
# 计算平均混淆矩阵（按列求平均）
avg_cm = np.sum(stacked_cm,axis=0).astype(int)

print(avg_cm)

cm_str = np.array2string(stacked_cm, separator=",")
print(cm_str)
print(type(cm_str))

0    [[51, 3], [12, 12]]
1     [[42, 8], [6, 22]]
2    [[44, 12], [5, 17]]
3     [[52, 7], [7, 12]]
4     [[47, 8], [4, 19]]
5    [[55, 3], [10, 10]]
6      [[57, 4], [7, 9]]
7    [[42, 6], [10, 19]]
8     [[37, 8], [2, 30]]
9     [[40, 9], [4, 24]]
Name: cm, dtype: object
[[467  68]
 [ 67 174]]
[534 242]
[[467, 68],
 [ 67,174]]
<class 'str'>


In [6]:

test_df = test_df.drop(columns=["epoch", "loss", "cm"])
test_df = test_df.set_index("subject")
test_df = test_df.T
# test_df.reset_index(inplace=True)

# 两行分别为acc和f1，将两行结果归并到一行，以/分割
test_df = test_df.apply(lambda x: x.apply(lambda x: "{:.2f}".format(x)))
print(test_df.index)
print(test_df.columns)

# 合并两行数据
combined_row = test_df.loc['acc'].astype(str) + '/' + test_df.loc['f1-score'].astype(str)
# 创建新的 DataFrame，更新 index 为 'acc/f1-score'
new_df = pd.DataFrame([combined_row], index=['acc/f1-score'])
new_df.head()


Index(['acc', 'f1-score'], dtype='object')
Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 'Mean', 'Std'], dtype='object', name='subject')


subject,0,1,2,3,4,5,6,7,8,9,Mean,Std
acc/f1-score,0.81/0.79,0.82/0.82,0.78/0.79,0.82/0.82,0.85/0.85,0.83/0.82,0.86/0.85,0.79/0.79,0.87/0.87,0.83/0.83,0.83/0.82,0.00/0.00


In [8]:
# yaml文件
import os
import sys
import yaml
from pathlib import Path
import pandas as pd
def add_project_root_to_sys_path():
    """动态添加项目根目录到 sys.path"""
    project_root = Path.cwd().resolve().parent
    print(project_root)
    if str(project_root) not in sys.path:
        sys.path.append(str(project_root))

add_project_root_to_sys_path()

from argparse import ArgumentParser
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR

from pathlib import Path
from common.utils import (
    load_config,
    seed_all,
    normalize_cm,
)  # 假设有一个工具函数来加载配置文件
from main import parse_args, modify_config

config_path = f"{Path.cwd().parent}/config/config.yaml"
config = load_config(config_path)
args = parse_args(args=[])
print(args)

config = modify_config(config, args)
print(config)


/mnt/nvme1/yihaoyuan/Raven/RavenEx/multimodal_emotion_recognition
Namespace(checkpoint=None, config='/mnt/nvme1/yihaoyuan/Raven/RavenEx/multimodal_emotion_recognition/notebooks/config/config.yaml', data='Ruiwen', dependent=None, infer_input=None, mode='train', model='MFAFESM', num_classes=None, resume=False, using_modality=None)
[960, 41, 119]
['eeg', 'eye', 'au']
{'model': {'feature_extract': {'input_dim': 75, 'hidden_dim': 160, 'tok': 0.5}, 'feature_align': {'input_size': [960, 41, 119], 'embed_dim': 160, 'seq_len': 10}, 'fusion': {'embed_dim': 160, 'num_heads': 8, 'd_model': 640}, 'attention_encoder': {'num_layers': 6, 'd_model': 640, 'num_heads': 8, 'd_ff': 2048, 'dropout': 0.1, 'embed_dim': 160}, 'classifier': {'nb_classes': 4, 'embed_dim': 1600}}, 'training': {'ex_name': 'First Fire', 'batch_size': 64, 'epochs': 5, 'learning_rate': 0.0001, 'weight_decay': 0.002, 'optimizer': 'adam', 'loss_function': 'cross_entropy', 'dependent': False, 'n_folds': 10}, 'data': {'data_path': '/data

In [9]:
# {'model': {'feature_extract': {'input_dim': 75, 'hidden_dim': 160, 'tok': 0.5}, 'feature_align': {'input_size': [960, 41, 119], 'embed_dim': 160, 'seq_len': 10}, 'fusion': {'embed_dim': 160, 'num_heads': 8, 'd_model': 160}, 'attention_encoder': {'num_layers': 6, 'd_model': 160, 'num_heads': 8, 'd_ff': 2048, 'dropout': 0.1, 'embed_dim': 160}, 'classifier': {'nb_classes': 4, 'embed_dim': 1600}}, 'training': {'ex_name': 'First Fire', 'batch_size': 64, 'epochs': 300, 'learning_rate': 0.0001, 'weight_decay': 0.002, 'optimizer': 'adam', 'loss_function': 'cross_entropy', 'dependent': False, 'n_folds': 10}, 'data': {'data_path': '/data/Ruiwen/data_with_ICA.pkl', 'subject_lists': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34], 'modalities': ['eeg', 'eye', 'au'], 'input_size': [960, 41, 119], 'num_workers': 4, 'ch_nums': 31, 'ex_nums': 48}, 'logging': {'log_dir': '/mnt/nvme1/yihaoyuan/Raven/RavenEx/multimodal_emotion_recognition/logs/independent', 'model_dir': '/mnt/nvme1/yihaoyuan/Raven/RavenEx/multimodal_emotion_recognition/outputs/independent', 'save_best_only': True}, 'device': {'gpu': True, 'gpu_ids': [0]}, 'seed': 42, 'num_classes': 4}

# 递归方式拆解字典，将嵌套字典的key拼接起来形成单个key
def dict_format(dic, parent_key=""):
    """
    递归方式拆解字典，将嵌套字典的key拼接起来形成单个key
    """
    items = []
    for k, v in dic.items():
        new_key = f"{parent_key}.{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(dict_format(v, new_key).items())
        elif isinstance(v, list):
            items.append((new_key, ",".join([str(i) for i in v])))
        else:
            items.append((new_key, v))
    return dict(items)

new_config = dict_format(config)
print(new_config)

{'model.feature_extract.input_dim': 75, 'model.feature_extract.hidden_dim': 160, 'model.feature_extract.tok': 0.5, 'model.feature_align.input_size': '960,41,119', 'model.feature_align.embed_dim': 160, 'model.feature_align.seq_len': 10, 'model.fusion.embed_dim': 160, 'model.fusion.num_heads': 8, 'model.fusion.d_model': 640, 'model.attention_encoder.num_layers': 6, 'model.attention_encoder.d_model': 640, 'model.attention_encoder.num_heads': 8, 'model.attention_encoder.d_ff': 2048, 'model.attention_encoder.dropout': 0.1, 'model.attention_encoder.embed_dim': 160, 'model.classifier.nb_classes': 4, 'model.classifier.embed_dim': 1600, 'training.ex_name': 'First Fire', 'training.batch_size': 64, 'training.epochs': 5, 'training.learning_rate': 0.0001, 'training.weight_decay': 0.002, 'training.optimizer': 'adam', 'training.loss_function': 'cross_entropy', 'training.dependent': False, 'training.n_folds': 10, 'data.data_path': '/data/Ruiwen/data_with_ICA.pkl', 'data.subject_lists': '2,3,4,5,6,7,8,

In [10]:
config_df = pd.DataFrame(new_config, index=[0])
config_df.head()

,model.feature_extract.input_dim,model.feature_extract.hidden_dim,model.feature_extract.tok,model.feature_align.input_size,model.feature_align.embed_dim,model.feature_align.seq_len,model.fusion.embed_dim,model.fusion.num_heads,model.fusion.d_model,model.attention_encoder.num_layers,...,data.num_workers,data.ch_nums,data.ex_nums,logging.log_dir,logging.model_dir,logging.save_best_only,device.gpu,device.gpu_ids,seed,num_classes
0,75,160,0.5,"960,41,119",160,10,160,8,640,6,...,4,31,48,/mnt/nvme1/yihaoyuan/Raven/RavenEx/multimodal_...,/mnt/nvme1/yihaoyuan/Raven/RavenEx/multimodal_...,True,True,0,42,4


In [11]:
new_df.head()

subject,0,1,2,3,4,5,6,7,8,9,Mean,Std
acc/f1-score,0.81/0.79,0.82/0.82,0.78/0.79,0.82/0.82,0.85/0.85,0.83/0.82,0.86/0.85,0.79/0.79,0.87/0.87,0.83/0.83,0.83/0.82,0.00/0.00


In [18]:
# 将两个 DataFrame 横向合并
new_df = new_df.reset_index(drop=True)
# 将cm放在最后一列
config_df["cm"] = cm_str

final_df = pd.concat([config_df, new_df], axis=1)
final_df.head()

,model.feature_extract.input_dim,model.feature_extract.hidden_dim,model.feature_extract.tok,model.feature_align.input_size,model.feature_align.embed_dim,model.feature_align.seq_len,model.fusion.embed_dim,model.fusion.num_heads,model.fusion.d_model,model.attention_encoder.num_layers,...,2,3,4,5,6,7,8,9,Mean,Std
0,75,160,0.5,"960,41,119",160,10,160,8,640,6,...,0.78/0.79,0.82/0.82,0.85/0.85,0.83/0.82,0.86/0.85,0.79/0.79,0.87/0.87,0.83/0.83,0.83/0.82,0.00/0.00


In [1]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize
import numpy as np

# 假设有3个类别（0, 1, 2）
y_true = np.array([0, 1, 2, 2, 1, 0])  # 真实标签
y_score = np.array([[0.8, 0.1, 0.1],  # 对应的每个样本的预测概率
                    [0.2, 0.7, 0.1],
                    [0.3, 0.2, 0.5],
                    [0.1, 0.2, 0.7],
                    [0.4, 0.5, 0.1],
                    [0.9, 0.05, 0.05]])

# 将标签进行二值化
y_true_bin = label_binarize(y_true, classes=[0, 1, 2])

# 计算多分类的 AUC，使用 "macro" 计算所有类别AUC的均值
auc_macro = roc_auc_score(y_true_bin, y_score, average='macro', multi_class='ovr')
print(f"Macro AUC: {auc_macro}")

# 计算 Micro AUC
auc_micro = roc_auc_score(y_true_bin, y_score, average='micro', multi_class='ovr')
print(f"Micro AUC: {auc_micro}")


Macro AUC: 1.0
Micro AUC: 0.9999999999999999


: 